In [94]:
# imports
# system related
import sys
import subprocess

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep

# snap7 and logging
import snap7 as s7


print('imports succesful')

imports succesful


## Connecting to plc

In [95]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

connection status: True


## Defining get & set functions for plc variables

In [96]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


## Creating RPMs to loop thru

In [109]:
# creating random values to loop thru

############ CHECK THESE VAls!!

rpm_choices = [10, 13, 16 , 19]
num_values = 200
rpm_interval = 5  # s
T_choices = list(range(280, 285, 5))
print(T_choices)

rpms = [rpm_choices[0]]

def count_elements(seq):
        """Tally elements from `seq`."""
        lst = [0,0,0,0,0,0,0]
        for i in seq:
            j = int(i/5)
            lst[j+3] += 1
        lst.pop(3)
        return lst


a = 0.35
b=0.5-a
print('')

while len(rpms) < num_values:
    # create random rpm value

    rpm = np.random.choice(rpm_choices, 1, p=[a, b, b, a])[0]
    # append if difference is high enough. 
    if rpm != rpms[-1]:
        rpms.append(rpm)

print(rpms)




[280]

[10, 16, 10, 16, 19, 10, 16, 10, 19, 16, 19, 10, 19, 10, 19, 16, 10, 19, 10, 16, 10, 16, 10, 13, 19, 13, 19, 16, 10, 19, 10, 19, 10, 19, 10, 19, 13, 10, 13, 16, 19, 16, 19, 13, 19, 16, 19, 10, 19, 13, 19, 16, 10, 19, 13, 10, 13, 16, 13, 19, 13, 10, 19, 10, 16, 13, 10, 19, 10, 19, 16, 10, 19, 10, 19, 10, 19, 16, 19, 10, 19, 10, 19, 13, 10, 19, 13, 19, 16, 13, 16, 19, 10, 19, 16, 13, 10, 19, 10, 19, 10, 19, 10, 16, 13, 19, 10, 19, 10, 13, 19, 10, 13, 19, 10, 16, 10, 19, 10, 16, 10, 19, 10, 19, 13, 19, 10, 19, 10, 19, 10, 19, 16, 13, 16, 10, 16, 19, 13, 10, 19, 10, 16, 10, 13, 19, 13, 19, 13, 10, 13, 19, 10, 19, 16, 10, 19, 16, 13, 10, 19, 10, 13, 16, 19, 16, 10, 13, 10, 13, 10, 19, 10, 13, 16, 10, 16, 13, 19, 10, 19, 10, 19, 10, 16, 13, 10, 13, 19, 10, 13, 16, 10, 19, 10, 13, 19, 13, 10, 19]


## Defining DB number and start offsets for the variables

In [97]:
db_py = 50

# defining start_offsets (so) for the variables in the DB50
so_rpm = 0
so_hz1 = 4
so_hz2 = 8
so_hz3 = 12
so_hz4 = 16


In [111]:
# for loop true random rpms
tot = len(rpms)*len(T_choices)
try:
    for j, T in enumerate(T_choices):
        set_real(db_py, so_hz1, T)
        set_real(db_py, so_hz2, T)
        set_real(db_py, so_hz3, T)
        set_real(db_py, so_hz4, T)
        for i, rpm in enumerate(rpms):
            prog=j*len(rpms)+i
            set_real(db_py, so_rpm, rpm)
            print(f'Progress: |{int(100*prog/tot)*"█" + int(100*(1-(prog/tot)))*"-"} | {np.round(prog/tot*100 ,1)}% | {prog}/{tot} | ETA: {np.round((tot-prog)*rpm_interval/60,1)} min | RPM: {rpm} | T: {j+1}/{len(T_choices)} | T_set = {T}  ' , end='\r')
            sleep(rpm_interval)
        
except KeyboardInterrupt:
    print('Sending RPMs interupted by keystroke                                                         It came this far: ')

print('all rpms have been sent, the job is finished')

Sending RPMs interupted by keystroke                                                         It came this far: | 89.0% | 178/200 | ETA: 1.8 min | RPM: 19 | T: 1/1 | T_set = 280   
all rpms have been sent, the job is finished


In [98]:
set_real(db_py, so_hz1, 250)
set_real(db_py, so_hz2, 250)
set_real(db_py, so_hz3, 250)
set_real(db_py, so_hz4, 250)